***GENERATED CODE FOR homecreditriskprediction PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NAME_CONTRACT_TYPE', 'transformation_label': 'String Indexer'}], 'feature': 'NAME_CONTRACT_TYPE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '242', 'mean': '', 'stddev': '', 'min': 'Cash loans', 'max': 'Revolving loans', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NAME_CONTRACT_TYPE'}, {'feature_label': 'NAME_CONTRACT_TYPE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NAME_CONTRACT_TYPE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CODE_GENDER', 'transformation_label': 'String Indexer'}], 'feature': 'CODE_GENDER', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CODE_GENDER'}, {'feature_label': 'CODE_GENDER', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CODE_GENDER')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FLAG_OWN_CAR', 'transformation_label': 'String Indexer'}], 'feature': 'FLAG_OWN_CAR', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'Y', 'max': 'Y', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FLAG_OWN_CAR'}, {'feature_label': 'FLAG_OWN_CAR', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FLAG_OWN_CAR')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FLAG_OWN_REALTY', 'transformation_label': 'String Indexer'}], 'feature': 'FLAG_OWN_REALTY', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'N', 'max': 'Y', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FLAG_OWN_REALTY'}, {'feature_label': 'FLAG_OWN_REALTY', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FLAG_OWN_REALTY')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AMT_INCOME_TOTAL', 'threshold': 235351.95247933886, 'transformation_label': 'Binarizer'}], 'feature': 'AMT_INCOME_TOTAL', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '242', 'mean': '235351.95', 'stddev': '155987.41', 'min': '40500.0', 'max': '1215000.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'AMT_INCOME_TOTAL'}, {'feature_label': 'AMT_INCOME_TOTAL', 'threshold': 235351.95247933886, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('AMT_INCOME_TOTAL')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AMT_CREDIT', 'threshold': 702896.132231405, 'transformation_label': 'Binarizer'}], 'feature': 'AMT_CREDIT', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '242', 'mean': '702896.13', 'stddev': '474699.32', 'min': '67765.5', 'max': '2695500.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'AMT_CREDIT'}, {'feature_label': 'AMT_CREDIT', 'threshold': 702896.132231405, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('AMT_CREDIT')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AMT_ANNUITY', 'threshold': 32517.464876033056, 'transformation_label': 'Binarizer'}], 'feature': 'AMT_ANNUITY', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '242', 'mean': '32517.46', 'stddev': '17618.8', 'min': '4504.5', 'max': '110488.5', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'AMT_ANNUITY'}, {'feature_label': 'AMT_ANNUITY', 'threshold': 32517.464876033056, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('AMT_ANNUITY')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AMT_GOODS_PRICE', 'threshold': 635954.6900826446, 'transformation_label': 'Binarizer'}], 'feature': 'AMT_GOODS_PRICE', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '242', 'mean': '635954.69', 'stddev': '433617.68', 'min': '58500.0', 'max': '2250000.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'AMT_GOODS_PRICE'}, {'feature_label': 'AMT_GOODS_PRICE', 'threshold': 635954.6900826446, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('AMT_GOODS_PRICE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NAME_TYPE_SUITE', 'transformation_label': 'String Indexer'}], 'feature': 'NAME_TYPE_SUITE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'Family', 'max': 'Unaccompanied', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NAME_TYPE_SUITE'}, {'feature_label': 'NAME_TYPE_SUITE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NAME_TYPE_SUITE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NAME_INCOME_TYPE', 'transformation_label': 'String Indexer'}], 'feature': 'NAME_INCOME_TYPE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'Commercial associate', 'max': 'Working', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NAME_INCOME_TYPE'}, {'feature_label': 'NAME_INCOME_TYPE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NAME_INCOME_TYPE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NAME_EDUCATION_TYPE', 'transformation_label': 'String Indexer'}], 'feature': 'NAME_EDUCATION_TYPE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'Academic degree', 'max': 'Secondary / secondary special', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NAME_EDUCATION_TYPE'}, {'feature_label': 'NAME_EDUCATION_TYPE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NAME_EDUCATION_TYPE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NAME_FAMILY_STATUS', 'transformation_label': 'String Indexer'}], 'feature': 'NAME_FAMILY_STATUS', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'Civil marriage', 'max': 'Widow', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NAME_FAMILY_STATUS'}, {'feature_label': 'NAME_FAMILY_STATUS', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NAME_FAMILY_STATUS')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NAME_HOUSING_TYPE', 'transformation_label': 'String Indexer'}], 'feature': 'NAME_HOUSING_TYPE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'House / apartment', 'max': 'With parents', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NAME_HOUSING_TYPE'}, {'feature_label': 'NAME_HOUSING_TYPE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NAME_HOUSING_TYPE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OCCUPATION_TYPE', 'transformation_label': 'String Indexer'}], 'feature': 'OCCUPATION_TYPE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'Accountants', 'max': 'Waiters/barmen staff', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OCCUPATION_TYPE'}, {'feature_label': 'OCCUPATION_TYPE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OCCUPATION_TYPE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'WEEKDAY_APPR_PROCESS_START', 'transformation_label': 'String Indexer'}], 'feature': 'WEEKDAY_APPR_PROCESS_START', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'FRIDAY', 'max': 'WEDNESDAY', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'WEEKDAY_APPR_PROCESS_STAR...'}, {'feature_label': 'WEEKDAY_APPR_PROCESS_START', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'WEEKDAY_APPR_PROCESS_START')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ORGANIZATION_TYPE', 'transformation_label': 'String Indexer'}], 'feature': 'ORGANIZATION_TYPE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'Bank', 'max': 'University', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ORGANIZATION_TYPE'}, {'feature_label': 'ORGANIZATION_TYPE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ORGANIZATION_TYPE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FONDKAPREMONT_MODE', 'transformation_label': 'String Indexer'}], 'feature': 'FONDKAPREMONT_MODE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'not specified', 'max': 'reg oper spec account', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FONDKAPREMONT_MODE'}, {'feature_label': 'FONDKAPREMONT_MODE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FONDKAPREMONT_MODE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'HOUSETYPE_MODE', 'transformation_label': 'String Indexer'}], 'feature': 'HOUSETYPE_MODE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'block of flats', 'max': 'terraced house', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'HOUSETYPE_MODE'}, {'feature_label': 'HOUSETYPE_MODE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('HOUSETYPE_MODE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'WALLSMATERIAL_MODE', 'transformation_label': 'String Indexer'}], 'feature': 'WALLSMATERIAL_MODE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'Block', 'max': 'Wooden', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'WALLSMATERIAL_MODE'}, {'feature_label': 'WALLSMATERIAL_MODE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('WALLSMATERIAL_MODE')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'EMERGENCYSTATE_MODE', 'transformation_label': 'String Indexer'}], 'feature': 'EMERGENCYSTATE_MODE', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '242', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'EMERGENCYSTATE_MODE'}, {'feature_label': 'EMERGENCYSTATE_MODE', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('EMERGENCYSTATE_MODE')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
#%run homecreditriskpredictionHooks.ipynb
try:
	#sourcePreExecutionHook()

	applicationtrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1614103605257_application_train.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line', 'FilePath': '/Banking Models/Loan Repayments Model/application_train.csv', 'filename': '1614103605257_application_train.csv', 'viewFileName': 'application_train.csv'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run homecreditriskpredictionHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(applicationtrain,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "SK_ID_CURR", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "195237.52", "stddev": "53298.98", "min": "100343", "max": "292571", "missing": "0"}, "updatedLabel": "SK_ID_CURR"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TARGET", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.05", "stddev": "0.23", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "TARGET"}, {"transformationsData": [{"feature_label": "NAME_CONTRACT_TYPE", "transformation_label": "String Indexer"}], "feature": "NAME_CONTRACT_TYPE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "Cash loans", "max": "Revolving loans", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NAME_CONTRACT_TYPE"}, {"transformationsData": [{"feature_label": "CODE_GENDER", "transformation_label": "String Indexer"}], "feature": "CODE_GENDER", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CODE_GENDER"}, {"transformationsData": [{"feature_label": "FLAG_OWN_CAR", "transformation_label": "String Indexer"}], "feature": "FLAG_OWN_CAR", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "Y", "max": "Y", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FLAG_OWN_CAR"}, {"transformationsData": [{"feature_label": "FLAG_OWN_REALTY", "transformation_label": "String Indexer"}], "feature": "FLAG_OWN_REALTY", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "N", "max": "Y", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FLAG_OWN_REALTY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CNT_CHILDREN", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.57", "stddev": "0.73", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "CNT_CHILDREN"}, {"transformationsData": [{"feature_label": "AMT_INCOME_TOTAL", "threshold": 235351.95247933886, "transformation_label": "Binarizer"}], "feature": "AMT_INCOME_TOTAL", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "235351.95", "stddev": "155987.41", "min": "40500.0", "max": "1215000.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "AMT_INCOME_TOTAL"}, {"transformationsData": [{"feature_label": "AMT_CREDIT", "threshold": 702896.132231405, "transformation_label": "Binarizer"}], "feature": "AMT_CREDIT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "702896.13", "stddev": "474699.32", "min": "67765.5", "max": "2695500.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "AMT_CREDIT"}, {"transformationsData": [{"feature_label": "AMT_ANNUITY", "threshold": 32517.464876033056, "transformation_label": "Binarizer"}], "feature": "AMT_ANNUITY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "32517.46", "stddev": "17618.8", "min": "4504.5", "max": "110488.5", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "AMT_ANNUITY"}, {"transformationsData": [{"feature_label": "AMT_GOODS_PRICE", "threshold": 635954.6900826446, "transformation_label": "Binarizer"}], "feature": "AMT_GOODS_PRICE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "635954.69", "stddev": "433617.68", "min": "58500.0", "max": "2250000.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "AMT_GOODS_PRICE"}, {"transformationsData": [{"feature_label": "NAME_TYPE_SUITE", "transformation_label": "String Indexer"}], "feature": "NAME_TYPE_SUITE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "Family", "max": "Unaccompanied", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NAME_TYPE_SUITE"}, {"transformationsData": [{"feature_label": "NAME_INCOME_TYPE", "transformation_label": "String Indexer"}], "feature": "NAME_INCOME_TYPE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "Commercial associate", "max": "Working", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NAME_INCOME_TYPE"}, {"transformationsData": [{"feature_label": "NAME_EDUCATION_TYPE", "transformation_label": "String Indexer"}], "feature": "NAME_EDUCATION_TYPE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "Academic degree", "max": "Secondary / secondary special", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NAME_EDUCATION_TYPE"}, {"transformationsData": [{"feature_label": "NAME_FAMILY_STATUS", "transformation_label": "String Indexer"}], "feature": "NAME_FAMILY_STATUS", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "Civil marriage", "max": "Widow", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NAME_FAMILY_STATUS"}, {"transformationsData": [{"feature_label": "NAME_HOUSING_TYPE", "transformation_label": "String Indexer"}], "feature": "NAME_HOUSING_TYPE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "House / apartment", "max": "With parents", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NAME_HOUSING_TYPE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "REGION_POPULATION_RELATIVE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.02", "stddev": "0.02", "min": "0.002042", "max": "0.072508", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "REGION_POPULATION_RELATIV..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DAYS_BIRTH", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "-14544.41", "stddev": "3394.57", "min": "-23763", "max": "-8368", "missing": "0"}, "updatedLabel": "DAYS_BIRTH"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DAYS_EMPLOYED", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "-2243.81", "stddev": "2083.41", "min": "-12659", "max": "-91", "missing": "0"}, "updatedLabel": "DAYS_EMPLOYED"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DAYS_REGISTRATION", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "-4507.55", "stddev": "3141.02", "min": "-12498", "max": "-2", "missing": "0"}, "updatedLabel": "DAYS_REGISTRATION"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DAYS_ID_PUBLISH", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "-2933.37", "stddev": "1584.07", "min": "-6070", "max": "-5", "missing": "0"}, "updatedLabel": "DAYS_ID_PUBLISH"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OWN_CAR_AGE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "11.3", "stddev": "11.51", "min": "0", "max": "65", "missing": "0"}, "updatedLabel": "OWN_CAR_AGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_MOBIL", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_MOBIL"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_EMP_PHONE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "1.0", "stddev": "0.06", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_EMP_PHONE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_WORK_PHONE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.21", "stddev": "0.41", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_WORK_PHONE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_CONT_MOBILE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "1.0", "stddev": "0.06", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_CONT_MOBILE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_PHONE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.34", "stddev": "0.47", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_PHONE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_EMAIL", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.14", "stddev": "0.35", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_EMAIL"}, {"transformationsData": [{"feature_label": "OCCUPATION_TYPE", "transformation_label": "String Indexer"}], "feature": "OCCUPATION_TYPE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "Accountants", "max": "Waiters/barmen staff", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OCCUPATION_TYPE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CNT_FAM_MEMBERS", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "2.35", "stddev": "0.89", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "CNT_FAM_MEMBERS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "REGION_RATING_CLIENT", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "2.03", "stddev": "0.57", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "REGION_RATING_CLIENT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "REGION_RATING_CLIENT_W_CITY", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "1.96", "stddev": "0.52", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "REGION_RATING_CLIENT_W_CI..."}, {"transformationsData": [{"feature_label": "WEEKDAY_APPR_PROCESS_START", "transformation_label": "String Indexer"}], "feature": "WEEKDAY_APPR_PROCESS_START", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "FRIDAY", "max": "WEDNESDAY", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "WEEKDAY_APPR_PROCESS_STAR..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "HOUR_APPR_PROCESS_START", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "12.33", "stddev": "3.3", "min": "3", "max": "20", "missing": "0"}, "updatedLabel": "HOUR_APPR_PROCESS_START"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "REG_REGION_NOT_LIVE_REGION", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "REG_REGION_NOT_LIVE_REGIO..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "REG_REGION_NOT_WORK_REGION", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.08", "stddev": "0.28", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "REG_REGION_NOT_WORK_REGIO..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LIVE_REGION_NOT_WORK_REGION", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.05", "stddev": "0.22", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "LIVE_REGION_NOT_WORK_REGI..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "REG_CITY_NOT_LIVE_CITY", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.04", "stddev": "0.2", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "REG_CITY_NOT_LIVE_CITY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "REG_CITY_NOT_WORK_CITY", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.16", "stddev": "0.37", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "REG_CITY_NOT_WORK_CITY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LIVE_CITY_NOT_WORK_CITY", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.12", "stddev": "0.33", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "LIVE_CITY_NOT_WORK_CITY"}, {"transformationsData": [{"feature_label": "ORGANIZATION_TYPE", "transformation_label": "String Indexer"}], "feature": "ORGANIZATION_TYPE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "Bank", "max": "University", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ORGANIZATION_TYPE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EXT_SOURCE_1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.51", "stddev": "0.21", "min": "0.054382474", "max": "0.894367434", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "EXT_SOURCE_1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EXT_SOURCE_2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.56", "stddev": "0.17", "min": "0.00353192", "max": "0.802276298", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "EXT_SOURCE_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EXT_SOURCE_3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.49", "stddev": "0.2", "min": "0.039510522", "max": "0.852828467", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "EXT_SOURCE_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "APARTMENTS_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.13", "stddev": "0.1", "min": "0.0", "max": "0.6392", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "APARTMENTS_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BASEMENTAREA_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.09", "stddev": "0.06", "min": "0.0", "max": "0.4529", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BASEMENTAREA_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YEARS_BEGINEXPLUATATION_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.98", "stddev": "0.01", "min": "0.9473", "max": "0.9995", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "YEARS_BEGINEXPLUATATION_A..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YEARS_BUILD_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.76", "stddev": "0.11", "min": "0.2792", "max": "0.9932", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "YEARS_BUILD_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "COMMONAREA_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.05", "stddev": "0.06", "min": "0.0", "max": "0.3667", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "COMMONAREA_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ELEVATORS_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.09", "stddev": "0.13", "min": "0.0", "max": "0.76", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ELEVATORS_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ENTRANCES_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.15", "stddev": "0.09", "min": "0.0", "max": "0.6552", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ENTRANCES_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLOORSMAX_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.25", "stddev": "0.14", "min": "0.0", "max": "0.9375", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "FLOORSMAX_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLOORSMIN_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.25", "stddev": "0.16", "min": "0.0", "max": "0.7917", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "FLOORSMIN_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LANDAREA_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.07", "stddev": "0.08", "min": "0.0", "max": "0.8873", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LANDAREA_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LIVINGAPARTMENTS_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.1", "stddev": "0.08", "min": "0.0", "max": "0.5119", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LIVINGAPARTMENTS_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LIVINGAREA_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.12", "stddev": "0.1", "min": "0.0012", "max": "0.7951", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LIVINGAREA_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NONLIVINGAPARTMENTS_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.01", "stddev": "0.02", "min": "0.0", "max": "0.2085", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NONLIVINGAPARTMENTS_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NONLIVINGAREA_AVG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.02", "stddev": "0.04", "min": "0.0", "max": "0.2525", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NONLIVINGAREA_AVG"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "APARTMENTS_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.12", "stddev": "0.1", "min": "0.0", "max": "0.6513", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "APARTMENTS_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BASEMENTAREA_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.09", "stddev": "0.07", "min": "0.0", "max": "0.47", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BASEMENTAREA_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YEARS_BEGINEXPLUATATION_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.98", "stddev": "0.01", "min": "0.9459", "max": "0.9995", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "YEARS_BEGINEXPLUATATION_M..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YEARS_BUILD_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.76", "stddev": "0.11", "min": "0.2878", "max": "0.9935", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "YEARS_BUILD_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "COMMONAREA_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.04", "stddev": "0.06", "min": "0.0", "max": "0.37", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "COMMONAREA_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ELEVATORS_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.08", "stddev": "0.12", "min": "0.0", "max": "0.7653", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ELEVATORS_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ENTRANCES_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.15", "stddev": "0.09", "min": "0.0", "max": "0.6552", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ENTRANCES_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLOORSMAX_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.24", "stddev": "0.14", "min": "0.0", "max": "0.875", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "FLOORSMAX_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLOORSMIN_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.24", "stddev": "0.15", "min": "0.0", "max": "0.7917", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "FLOORSMIN_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LANDAREA_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.06", "stddev": "0.08", "min": "0.0", "max": "0.9076", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LANDAREA_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LIVINGAPARTMENTS_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.11", "stddev": "0.09", "min": "0.0", "max": "0.5592", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LIVINGAPARTMENTS_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LIVINGAREA_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.11", "stddev": "0.1", "min": "0.0", "max": "0.8285", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LIVINGAREA_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NONLIVINGAPARTMENTS_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.01", "stddev": "0.02", "min": "0.0", "max": "0.2101", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NONLIVINGAPARTMENTS_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NONLIVINGAREA_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.02", "stddev": "0.04", "min": "0.0", "max": "0.2245", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NONLIVINGAREA_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "APARTMENTS_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.13", "stddev": "0.1", "min": "0.0", "max": "0.6454", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "APARTMENTS_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BASEMENTAREA_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.09", "stddev": "0.06", "min": "0.0", "max": "0.4529", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BASEMENTAREA_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YEARS_BEGINEXPLUATATION_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.98", "stddev": "0.01", "min": "0.9473", "max": "0.9995", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "YEARS_BEGINEXPLUATATION_M..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YEARS_BUILD_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.76", "stddev": "0.11", "min": "0.2888", "max": "0.9933", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "YEARS_BUILD_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "COMMONAREA_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.05", "stddev": "0.06", "min": "0.0", "max": "0.369", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "COMMONAREA_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ELEVATORS_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.08", "stddev": "0.13", "min": "0.0", "max": "0.76", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ELEVATORS_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ENTRANCES_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.15", "stddev": "0.09", "min": "0.0", "max": "0.6552", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ENTRANCES_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLOORSMAX_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.25", "stddev": "0.15", "min": "0.0", "max": "0.9375", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "FLOORSMAX_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLOORSMIN_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.25", "stddev": "0.16", "min": "0.0", "max": "0.7917", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "FLOORSMIN_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LANDAREA_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.07", "stddev": "0.08", "min": "0.0", "max": "0.9028", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LANDAREA_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LIVINGAPARTMENTS_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.1", "stddev": "0.08", "min": "0.0", "max": "0.5207", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LIVINGAPARTMENTS_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LIVINGAREA_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.12", "stddev": "0.1", "min": "0.0013", "max": "0.8094", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LIVINGAREA_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NONLIVINGAPARTMENTS_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.01", "stddev": "0.02", "min": "0.0", "max": "0.2096", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NONLIVINGAPARTMENTS_MEDI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NONLIVINGAREA_MEDI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.02", "stddev": "0.04", "min": "0.0", "max": "0.2578", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NONLIVINGAREA_MEDI"}, {"transformationsData": [{"feature_label": "FONDKAPREMONT_MODE", "transformation_label": "String Indexer"}], "feature": "FONDKAPREMONT_MODE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "not specified", "max": "reg oper spec account", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FONDKAPREMONT_MODE"}, {"transformationsData": [{"feature_label": "HOUSETYPE_MODE", "transformation_label": "String Indexer"}], "feature": "HOUSETYPE_MODE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "block of flats", "max": "terraced house", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "HOUSETYPE_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TOTALAREA_MODE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "242", "mean": "0.11", "stddev": "0.1", "min": "0.0", "max": "0.6364", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TOTALAREA_MODE"}, {"transformationsData": [{"feature_label": "WALLSMATERIAL_MODE", "transformation_label": "String Indexer"}], "feature": "WALLSMATERIAL_MODE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "Block", "max": "Wooden", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "WALLSMATERIAL_MODE"}, {"transformationsData": [{"feature_label": "EMERGENCYSTATE_MODE", "transformation_label": "String Indexer"}], "feature": "EMERGENCYSTATE_MODE", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "242", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "EMERGENCYSTATE_MODE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OBS_30_CNT_SOCIAL_CIRCLE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "1.3", "stddev": "2.19", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "OBS_30_CNT_SOCIAL_CIRCLE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DEF_30_CNT_SOCIAL_CIRCLE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.1", "stddev": "0.33", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "DEF_30_CNT_SOCIAL_CIRCLE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OBS_60_CNT_SOCIAL_CIRCLE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "1.29", "stddev": "2.17", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "OBS_60_CNT_SOCIAL_CIRCLE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DEF_60_CNT_SOCIAL_CIRCLE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.07", "stddev": "0.26", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "DEF_60_CNT_SOCIAL_CIRCLE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DAYS_LAST_PHONE_CHANGE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "-1143.97", "stddev": "881.64", "min": "-4131", "max": "0", "missing": "0"}, "updatedLabel": "DAYS_LAST_PHONE_CHANGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.65", "stddev": "0.48", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_4", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_5", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.02", "stddev": "0.14", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_6", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.06", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_7", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_8", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.22", "stddev": "0.42", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_8"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_9", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.06", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_9"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_10", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_10"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_11", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_11"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_12", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_12"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_13", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.02", "stddev": "0.16", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_13"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_14", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_14"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_15", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_15"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_16", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_16"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_17", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_17"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_18", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_18"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_19", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_19"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_20", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_20"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FLAG_DOCUMENT_21", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FLAG_DOCUMENT_21"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AMT_REQ_CREDIT_BUREAU_HOUR", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.02", "stddev": "0.14", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "AMT_REQ_CREDIT_BUREAU_HOU..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AMT_REQ_CREDIT_BUREAU_DAY", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.0", "stddev": "0.06", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "AMT_REQ_CREDIT_BUREAU_DAY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AMT_REQ_CREDIT_BUREAU_WEEK", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.04", "stddev": "0.2", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "AMT_REQ_CREDIT_BUREAU_WEE..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AMT_REQ_CREDIT_BUREAU_MON", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.35", "stddev": "0.82", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "AMT_REQ_CREDIT_BUREAU_MON"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AMT_REQ_CREDIT_BUREAU_QRT", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "0.25", "stddev": "0.6", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "AMT_REQ_CREDIT_BUREAU_QRT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AMT_REQ_CREDIT_BUREAU_YEAR", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "242", "mean": "1.98", "stddev": "1.82", "min": "0", "max": "8", "missing": "0"}, "updatedLabel": "AMT_REQ_CREDIT_BUREAU_YEA..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run homecreditriskpredictionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["SK_ID_CURR", "NAME_CONTRACT_TYPE", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", "NAME_TYPE_SUITE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "REGION_POPULATION_RELATIVE", "DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "OWN_CAR_AGE", "FLAG_MOBIL", "FLAG_EMP_PHONE", "FLAG_WORK_PHONE", "FLAG_CONT_MOBILE", "FLAG_PHONE", "FLAG_EMAIL", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS", "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY", "WEEKDAY_APPR_PROCESS_START", "HOUR_APPR_PROCESS_START", "REG_REGION_NOT_LIVE_REGION", "REG_REGION_NOT_WORK_REGION", "LIVE_REGION_NOT_WORK_REGION", "REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "LIVE_CITY_NOT_WORK_CITY", "ORGANIZATION_TYPE", "EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "APARTMENTS_AVG", "BASEMENTAREA_AVG", "YEARS_BEGINEXPLUATATION_AVG", "YEARS_BUILD_AVG", "COMMONAREA_AVG", "ELEVATORS_AVG", "ENTRANCES_AVG", "FLOORSMAX_AVG", "FLOORSMIN_AVG", "LANDAREA_AVG", "LIVINGAPARTMENTS_AVG", "LIVINGAREA_AVG", "NONLIVINGAPARTMENTS_AVG", "NONLIVINGAREA_AVG", "APARTMENTS_MODE", "BASEMENTAREA_MODE", "YEARS_BEGINEXPLUATATION_MODE", "YEARS_BUILD_MODE", "COMMONAREA_MODE", "ELEVATORS_MODE", "ENTRANCES_MODE", "FLOORSMAX_MODE", "FLOORSMIN_MODE", "LANDAREA_MODE", "LIVINGAPARTMENTS_MODE", "LIVINGAREA_MODE", "NONLIVINGAPARTMENTS_MODE", "NONLIVINGAREA_MODE", "APARTMENTS_MEDI", "BASEMENTAREA_MEDI", "YEARS_BEGINEXPLUATATION_MEDI", "YEARS_BUILD_MEDI", "COMMONAREA_MEDI", "ELEVATORS_MEDI", "ENTRANCES_MEDI", "FLOORSMAX_MEDI", "FLOORSMIN_MEDI", "LANDAREA_MEDI", "LIVINGAPARTMENTS_MEDI", "LIVINGAREA_MEDI", "NONLIVINGAPARTMENTS_MEDI", "NONLIVINGAREA_MEDI", "FONDKAPREMONT_MODE", "HOUSETYPE_MODE", "TOTALAREA_MODE", "WALLSMATERIAL_MODE", "EMERGENCYSTATE_MODE", "OBS_30_CNT_SOCIAL_CIRCLE", "DEF_30_CNT_SOCIAL_CIRCLE", "OBS_60_CNT_SOCIAL_CIRCLE", "DEF_60_CNT_SOCIAL_CIRCLE", "DAYS_LAST_PHONE_CHANGE", "FLAG_DOCUMENT_2", "FLAG_DOCUMENT_3", "FLAG_DOCUMENT_4", "FLAG_DOCUMENT_5", "FLAG_DOCUMENT_6", "FLAG_DOCUMENT_7", "FLAG_DOCUMENT_8", "FLAG_DOCUMENT_9", "FLAG_DOCUMENT_10", "FLAG_DOCUMENT_11", "FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13", "FLAG_DOCUMENT_14", "FLAG_DOCUMENT_15", "FLAG_DOCUMENT_16", "FLAG_DOCUMENT_17", "FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19", "FLAG_DOCUMENT_20", "FLAG_DOCUMENT_21", "AMT_REQ_CREDIT_BUREAU_HOUR", "AMT_REQ_CREDIT_BUREAU_DAY", "AMT_REQ_CREDIT_BUREAU_WEEK", "AMT_REQ_CREDIT_BUREAU_MON", "AMT_REQ_CREDIT_BUREAU_QRT", "AMT_REQ_CREDIT_BUREAU_YEAR", "NAME_CONTRACT_TYPE_stringindexer_transform", "CODE_GENDER_stringindexer_transform", "FLAG_OWN_CAR_stringindexer_transform", "FLAG_OWN_REALTY_stringindexer_transform", "AMT_INCOME_TOTAL_binarizer_transform", "AMT_CREDIT_binarizer_transform", "AMT_ANNUITY_binarizer_transform", "AMT_GOODS_PRICE_binarizer_transform", "NAME_TYPE_SUITE_stringindexer_transform", "NAME_INCOME_TYPE_stringindexer_transform", "NAME_EDUCATION_TYPE_stringindexer_transform", "NAME_FAMILY_STATUS_stringindexer_transform", "NAME_HOUSING_TYPE_stringindexer_transform", "OCCUPATION_TYPE_stringindexer_transform", "WEEKDAY_APPR_PROCESS_START_stringindexer_transform", "ORGANIZATION_TYPE_stringindexer_transform", "FONDKAPREMONT_MODE_stringindexer_transform", "HOUSETYPE_MODE_stringindexer_transform", "WALLSMATERIAL_MODE_stringindexer_transform", "EMERGENCYSTATE_MODE_stringindexer_transform"], "TARGET")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

